# Spatial and Temporal Correlation Analysis

This tutorial demonstrates how to map the statistical relationship between two spatio-temporal variables.

It also covers the following technical concepts:

* using THREDDS and [OPeNDAP](https://en.wikipedia.org/wiki/OPeNDAP)
* getting information on file dimensions
* subsetting netCDF data from OPeNDAP
* grid resampling
* linear and rank correlation and their statistical significance.

In [ ]:
# As usual, we start with our imports
import xarray as xr
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn
%matplotlib inline
seaborn.set_style('dark')

## Using THREDDS and OPeNDAP

One of the various benefits of netCDF format is that it can be made directly 
accessible over the internet (using a so-called OPeNDAP client) so that you 
can get a specific part of the data without having to download the entire file. 
Furthermore, you can connect to remotely accessible (so-called THREDDS) data 
catalogues, so that you can more easily view information on the data and get 
previews, before deciding whether to download. 

In this tutorial we will download some gridded data directly from the data 
catalogue that sits behind the [ausenv.online](http://ausenv.online) website. 
By way of introduction, make your way to the documentation on '_Downloading 
Data_' that you can find via the help (?) menu and button on the website. This 
should take you to [this web page](http://www.wenfo.org/wald/australias-environment/#Download).
Under the header '_Direct Links_' you will see two links for 
each data set: 

* *Direct download*: If you were to click on one of these links the browser 
will ask you where you want to put the file, or it may even start downloading 
without warning. This is the traditional way of getting data. Because gridded 
data files can be very large, this is only a good idea if you really want to 
use all the data (for example, if you are doing a national-scale analysis). 

* *THREDDS*: If you click on one of these links your browser should open a 
new web page with the NCI logo up top and then a whole lot of fairly unintelligible 
text and links. Unfortunately, that is the default look of a THREDDS catalog. 
There are ways of making it look more informative, and indeed 
[the official NCI Catalogue](https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/home)
does that. However, we won't need it right now, as we can find 
the information we want by clicking through to the link that appears behind 
the word OPENDAP. You will now see an OPeNDAP Data Access Form bunch of information 
on this data file, including (1) the Data URL, which we will be using; (2) Global 
Attributes, containing some metadata; (3) Information on the variables contained; 
their dimensions, size, units, etc.

With this information, we can write a few commands that directly gets the 
data we are interested in, without having to download the whole file first. 
To show that that is a major advantage, let's have a look at this 
([excerpted](http://www.wenfo.org/wald/australias-environment/#Download)):

> - **Tree cover** (annual mapping): 
    [direct download](http://dapds00.nci.org.au/thredds/fileServer/ub8/au/treecover/250m/ANUWALD.TreeCover.AllYears.250m.nc)  (>500 MB) or 
    [THREDDS](http://dap.nci.org.au/thredds/remoteCatalogService?command=subset&catalog=http://dapds00.nci.org.au/thredds/catalog/ub8/au/treecover/250m/catalog.xml&dataset=ub8-au/treecover/250m/ANUWALD.TreeCover.AllYears.250m.nc)

On the website you are warned that the entirely data file for Australia 
and all years is more than 500 MB, which is a waste of disk space if you are 
only interested in some part of Australia or a few years. Let's assume we are 
interested in tree cover for the same area near Canberra before and after the 
2003 fires, in 2002 and 2004, respectively. By following the THREDDS > OPENDAP 
link we can see the OPeNDAP Data Access Form and copy the Data URL:

In [ ]:
tree_data_url = 'http://dapds00.nci.org.au/thredds/dodsC/ub8/au/treecover/250m/ANUWALD.TreeCover.AllYears.250m.nc'

*At this point, we skip about a quarter of the Matlab tutorial, where you have to look up the order of the dimensions, calculate the pixel coordinates of the area you want to subset, and fiddle with a number of other tedious and error-prone steps.  Fortunately you are using Python, and the Xarray package does all of this for you.*

In [ ]:
tree_cover = xr.open_dataset(tree_data_url)
tree_cover

Note the shape of this data - it's more than ten thousand steps along each of the spatial dimensions!
As a general rule of thumb, this will make it too big to load into memory at once.

Let's check nbytes to see the total size.

In [ ]:
tree_cover.nbytes / 10 **9

*Here we dodge another chunk of the Matlab tutorial, where simply finding the data in a file is tricky.  Xarray's named dimensions make this much more pleasant!  We also get the benefit of "lazy loading", meaning we can read the whole file and* then *select the area we want, because the data is not fetched until it is needed.  In Matlab, you have to manaully calculate which part of the data you need before opening the dataset!*

Remember, to select all data inside a bounding box we select a `slice` from start to end, in coordinates.  Datetime coordinates [can be selected with partial strings](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#partial-string-indexing) in the form `YYYY-MM-DD`, and you can stop at any part.  In Xarray and in Pandas - which share their index and coordinate logic - a slice according to coordinates includes the endpoints.

In [ ]:
# in decimal degrees, taken from Tutorial 3
lat_bounds = slice(-35.250, -35.625)
lon_bounds = slice(148.750, 149.000)

# Note: this means "all times from the start of 2002 to the end of 2004".
# This includes 2003, but by coincidence there is no data for 2003!
time_bounds = slice('2002', '2004')

canberra_tree_cover = tree_cover.sel(
    latitude=lat_bounds, longitude=lon_bounds, time=time_bounds)
canberra_tree_cover

In [ ]:
# Let's check the size of our subset, so we know
# whether it's a good idea to download it into memory
canberra_tree_cover.nbytes

In [ ]:
# Since our subset is measured in KB, we'll download it now
canberra_tree_cover.load()
# And then draw some maps!
canberra_tree_cover.AllYears.plot.imshow(col='time', cmap='RdYlGn')

The reason for going back to this area and looking at forests was to make 
it easier for you to see if you got the correct part of the data cube. As a 
point of interest, if you compare the change from 2002 to 2004 above to the 
burn severity map you will see some similarities as well as differences. Mostly, 
this comes from the definition that is used here to map forests, as '_vegetation 
that has the potential to reach 2 metres height and more than 20% canopy cover_' 
(the definition used in the National Carbon Accounting System). The 'potential' 
bit has the interesting consequence that mapping of burnt forests needs to be 
adjusted back in time if there is evidence that the forest has regrown, because 
in retrospect it then evidently had the _potential_ to become forest. 

If you think this was confusing, you are not the only one - remember that this
definition is designed to facilitate payments or penalties for carbon farming
and land clearing, *not* as a biophysically meaningful summary.


## Mapping statistical relationships between variables

For a change of scenery, let's have a look at some different data, a different 
region, and different application. An exposed soil surface is vulnerable to 
wind and water erosion, and exposed soil can usually be considered in worse 
health than protected soil. Several government and semi-government bodies (e.g., 
catchment management authorities) have targets for soil surface cover protection, 
whether from living or dead vegetation (litter). As you can imagine, there is 
often a link between soil moisture and soil exposure, and this makes it harder 
to separate rainfall-related variations in soil exposure from long-term trends. 
Here, we will investigate this by:

- Finding the correlation between soil moisture and soil protective cover.
- Establishing a linear regression model to account for this relationship.
- Calculate the residual change in soil cover after accounting for the influence of soil moisture. 
- Interpreting the result in terms of land management.

To do that, we will use annual average soil moisture and soil exposure 
data from the Australia's Environment Explorer (http://ausenv.online). The soil 
exposure data are produced by CSIRO Land and Water based on MODIS remote sensing 
using the method published in 
[Guerschman et al. (2015)](http://www.sciencedirect.com/science/article/pii/S0034425715000395).
The soil moisture data are modelled (based on rainfall and other ground 
and satellite observations) by ANU using a version of the AWRA-L model
([Van Dijk, 2010](http://www.clw.csiro.au/publications/waterforahealthycountry/2010/wfhc-aus-water-resources-assessment-system.pdf))
(You can find these links in the netCDF metadata as well)

You will eventually need to decide on your region of interest. Here we will 
have a look at the greater Melbourne area, chosen because it has city, forests, 
pasture, cropland and water.  The bounds we will use are defined as:

**Remember: you can do this for any area you like - just come back later and set different bounds.** 

In [ ]:
# No time bounds this time - we'll use the whole timeseries
melb_lat_bounds = slice(-37.25, -38.15)
melb_lon_bounds = slice(144.45, 145.55)

In [ ]:
# Because we already know the bounds we want, we can open, select, load the data, and reorder lat/lon axes in one go
bare_soil = xr.open_dataset(
    'http://dapds00.nci.org.au/thredds/dodsC/ub8/au/FractCov/BS/FractCover.V3_0_1.AnnualMeans.aust.005.BS.nc')\
    .sel(latitude=melb_lat_bounds, longitude=melb_lon_bounds).load()\
    .AnnualMeans.transpose('time', 'latitude', 'longitude')
# Fix some names in the metadata - see what happens if you comment out these lines
bare_soil.name = 'bare_soil_fraction'
bare_soil.attrs = dict(short_name='bare_soil_fraction', long_name='Annual mean fraction of bare soil')
# It was stored as an integer, so divide by 100 to get fraction from percent
bare_soil /= 100
# And finally display the array
bare_soil

In [ ]:
bare_soil.isel(time=-1).plot.imshow(robust=True)

In [ ]:
# And we do the same thing for soil moisture ()
soil_moisture = xr.open_dataset(
    'http://dapds00.nci.org.au/thredds/dodsC/ub8/au/OzWALD/annual/OzWALD.annual.Ssoil.AnnualMeans.nc')\
    .sel(latitude=melb_lat_bounds, longitude=melb_lon_bounds).load()\
    .AnnualMeans.transpose('time', 'latitude', 'longitude')
soil_moisture.name = 'soil_moisture_mm'
soil_moisture.attrs['short_name'] = soil_moisture.name
soil_moisture

In [ ]:
soil_moisture.isel(time=-1).plot.imshow(robust=True)

At this point you may notice that the soil moisture data look a lot more 'blocky' 
(pixellated would be a more technical term) then the soil exposure data, meaning
that the spatial resolution of the former is several times lower than that of the latter.
This is a common occurrence when working with two gridded data sets. 

Because most analyses require arrays of the same shape (size and dimension), you will need
to resample in some way, choosing a target resolution and interpolation method.
A common choice is to resample to the shape of the predictand - in our case exposed 
soil fraction - or to choose the finest grid that is convenient to work with
(ie coarsening very large data).

[Iterpolating](https://en.wikipedia.org/wiki/Interpolation), or "zooming" an image
is a common operation - and a common source of misinterpreted data.  The two main
approaches are to fit a polynomial function to all the data points in a sequence,
or to fit a function *between* each point in such a way that they fit smoothly 
togther (called a "spline" - this is also the fundamental unit of vector data).
Unless a deep understanding of your specific data calls for something exotic,
and there are some very exotic methods that only work for specifc kinds of data,
spline interpolation is a very good choice.

The most important parameter is the *order* of the function(s) to fit.

- order=0 is nearest-neighbor; the new grid is still blocky but each grid consists of multiple pixels
- order=1 is linear; a poor choice for regular grids (interpolation works for irregular points too)
- order=3 is cubic; this is usually the best choice for images

See [wikipedia](https://en.wikipedia.org/wiki/Interpolation#In_higher_dimensions)
for an example of each type.  Of course it is also possible to use higher-order splines,
but they are only really useful on higher-dimensional (ie more than two) data.

We will use [`scipy.ndimage.zoom`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.zoom.html)
on each time-step of the soil moisture array.

In [ ]:
# TODO: use scipy.image.ndzoom to upscale soil moisture
from scipy.ndimage import zoom as ndzoom
help(ndzoom)

Now for the tricky bit: we will create a copy of our `bare_soil` data, replace the metadata, and finally fill it with the interpolated `soil_moisture` data.

In [ ]:
from copy import deepcopy

fine_moisture = deepcopy(bare_soil)
fine_moisture.attrs = soil_moisture.attrs
fine_moisture.name = soil_moisture.name
fine_moisture['time'] = soil_moisture.time

SPLINE_ORDER = 3  # Try some other values 0 to 5 for SPLINE_ORDER to see what happens
ZOOM_FACTOR = (len(bare_soil.latitude) / len(soil_moisture.latitude),
               len(bare_soil.longitude) / len(soil_moisture.longitude))

# It's also possible to zoom a 3D array by setting a factor of 1 for the time
# dimension, but this way we preserve the timesteps correctly.
for timestamp in fine_moisture.time:
    # Start by selecting the timestamp
    data = soil_moisture.sel(time=timestamp)
    # Then zoom to the desired scale, filling nodata values with zero so we can zoom
    output = ndzoom(np.nan_to_num(data), zoom=ZOOM_FACTOR, order=SPLINE_ORDER)
    # Assign output to the contents of the fine_moisture array
    fine = fine_moisture.sel(time=timestamp)
    fine[:] = output
    
    # Make sure the minimum is zero, so it remains physically plausible
    fine.values[fine.values < 0] = 0
    # Last, we'll copy both sets of NaN values so that we don't cause spurious correlations
    # Try commenting each of these out to see how the map changes!
    fine.values[np.isnan(bare_soil.sel(time=timestamp).values)] = np.nan  # from the high-res data
    fine.values[ndzoom(np.isnan(data), zoom=ZOOM_FACTOR, order=0)] = np.nan  # from low-res, with nearest (blocky) zooming

fine_moisture.isel(time=-1).plot.imshow(robust=True)

You will notice the data look rather different, which is a logical consequence 
of interpolation. However, you will also notice the effect of NaN values, which 
stuff up interpolation, and may start to appreciate that these missing data 
can be quite a pain.  (try changing how they are handled, as commented above)

Working out how to handle missing data when resampling can be tricky - for example,
maybe we should have filled them with the closest value to avoid skewing coastal
areas towards zero.  On the other hand, that would have skewed them a little high!
There is no universial rule for this - you just need good judgement, and to document
your methods so others can review and replicate your work.

## Correlation analysis

Now we have been able to load in the data and have made them of identical 
resolution, we can start to look at the actual correlation analysis. 

Recall once again [Anscombe's Quartet](https://en.wikipedia.org/wiki/Anscombe%27s_quartet) - 
before we calculate correlations, it is a good habit to draw some plots.

Let's start by picking a grid cell, and plotting the timeseries for each variable.

In [ ]:
# Try a few different values to see if the relationship holds
_lat, _lon = 100, 100

# To plot two lines on the same axes, we have to explicitly create and use a set of axes 
# For the second, `ax.twinx()` creates a clone of the axes with a shared x and independent y.
fig, ax = plt.subplots()
fine_moisture.isel(latitude=_lat, longitude=_lon).plot(ax=ax)
bare_soil.isel(latitude=_lat, longitude=_lon).plot(ax=ax.twinx(), color='red')

Fortunately, we do see the expected correlation - less moisture tends to mean more exposure.

Next, we'll prepare our data for the correlation analysis by converting it into a pair of one-dimensional Pandas series.  This allows us to do efficient pairwise comparisons and calculations.  Pandas data does not have coordinates, but does have an index - so we can still be confident that corresponding data in each series will be matched up.

In [ ]:
# Create an empty dataframe to hold our columns of data
df = pd.DataFrame()
# Convert each data array into a series, and add it to the dataframe
for data in [bare_soil, fine_moisture]:
    df[data.name] = data.to_series()
# Discard any rows with missing values - I would usually keep them,
# but we can't correlate anything with missing data
df = df.dropna()

# And examine the first five rows
df.head(5)

On the right, you can see our two data columns.  On the left, we have our index - 
which consists of three subcolumns.  This is called a [MultiIndex](http://pandas.pydata.org/pandas-docs/stable/advanced.html);
it works in the same way as a single-column index but allows us to group data by
or convert back to higher dimensions.

Xarray is fantastic for multidimensional gridded data, but Pandas is the standard
tool for data analysis in Python.  Now that we have a Pandas dataframe, we can use
all the general-purpose plotting and statistical tools too.

First, a simple scatter plot.  Scatter plots are unreadable once you have more than
a few thousand points, and running `df.size` will show that we have more than half a million rows of data.
We'll therefore take a random sample - meaning a differnt plot every time you run the cell!

In [ ]:
# Try sample numbers between 1 and 100,000; or even delete ".sample()"
df.sample(1000).plot.scatter(x='soil_moisture_mm', y='bare_soil_fraction')

### Seaborn, for statistical visualisation

We've imported [Seaborn](http://seaborn.pydata.org) into our notebook each week, 
and appreciated the nicer colour schemes - but now it's time to see where it really shines.

Matplotlib makes any plot possible; Pandas makes easy plots easy; and Seaborn is designed 
to make advanced statistical plots easy too - Xarray's support for drawing maps is
inspired by Seaborn, in fact!

For example, Seaborn makes it very easy to plot the joint distribution of two values with `jointplot`:

In [ ]:
seaborn.jointplot(
    x='soil_moisture_mm',
    y='bare_soil_fraction',
    data=df.sample(10000),
    # There are several ways to represent a join distribution.
    # Try un-commenting one kind at a time!
    #kind='hex', joint_kws=dict(gridsize=30),
    kind='kde', cmap='magma_r', n_levels=200,
    #kind='scatter',
)

The `kde`, or '[kernel density estimate](https://en.wikipedia.org/wiki/Kernel_density_estimation)', shows an estimate of the probabilities underlying our random sample.  The `hex` visualisation is an alternative to `scatter`, but less vulnerable to saturation where there are many similar values.

Seaborn has even calculated the correlation coefficient (pearsonr ~= 0.35) and p-value of this corrlation (very strong).  However while we can be highly confident that an overall correlation exists, soil moisture only explains R-squared (10 to 15 percent) of the variation in exposed soil fraction across the whole area.

If we had more columns in our dataframe, it would be equally easy to look at pairwise correlation and distributions using [`pairplot`](http://seaborn.pydata.org/generated/seaborn.pairplot.html).


### Linear correlation

In the last part of this notebook, we will use the `pearsonr` and `spearmanr` functions from SciPy
to calculate the correlation between soil moisture and exposed soil in the timeseries for each pixel.

The linear (or Pearson, or parametric) correlation coefficient is the most 
commonly measure the strength of the relationship between two variables. It 
is particularly well suited if both variables are close to normally distributed 
and a linear relationship can be assumed.  If the relationship seems non-linear,
then it would be better to do one of two things: 

- Calculate the rank (or Spearman, or non-parametric) correlation coefficient. 
  To calculate the rank correlation, you replace each _x-_value by the rank (or 
  index) it would have if all _x_-values are sorted in ascending order, then do 
  the same for _y_, and then calculate the normal ('linear') correlation between
  the resulting rank arrays.

- Transform either or both variables to make the relationship closer to linear, 
  for example taking the logarithm or square root of _x_ or _y_, or both.

In any event, it is always a good idea to always check whether the rank 
correlation is very different from the linear correlation coefficient. If the 
two approaches produce _R_- and _p_-values, that lead to similar conclusions, 
then that strengthens your analysis. However, if they produce different results, 
then that does not automatically mean that the relationship is not linear.

In [ ]:
from scipy.stats import linregress, pearsonr, spearmanr

In [ ]:
linregress(df.soil_moisture_mm, df.bare_soil_fraction)

In [ ]:
pearsonr(df.soil_moisture_mm, df.bare_soil_fraction)

In [ ]:
spearmanr(df.soil_moisture_mm, df.bare_soil_fraction)

You may notice that the ranked _R_-value is actually quite different from 
the linear _R_-value, given that we are working with half a million rows.
Looking at the plots above, the most likely reason is that the soil exposure 
data are not really (approximately) normally-distributed. Most values are between 
2 to 6 %, but while are some higher values, there are no lower values - logical, 
since soil exposure cannot be less than zero.

Here, you could argue whether the data should be transformed (perhaps using _log(y)_ ).
Instead, let's just calculate both types of correlation, and interpret them together.

In the next two cells, we will calculate the correlations in each and every cell, and make
a list of results which we then convert to an array.  *This is a very slow way to calculate anything*,
but it works even for functions that are difficult to express as operations along some 
axis of the array - so I'll use if for correlations, but not for mean, min, max, sum and so on.
Expect them to take two or three minutes!

In [ ]:
# Start by setting up a new dataset, with empty arrays along latitude and longitude
dims = ('latitude', 'longitude')
coords = {d: bare_soil[d] for d in dims}
correlation_data = {
    name: xr.DataArray(data=np.ndarray([len(bare_soil[d]) for d in dims]),
                       name=name, dims=dims)
    for name in 'pearson_r pearson_p spearman_r spearman_p'.split()
}
corr = xr.Dataset(data_vars=correlation_data, coords=coords)
corr

In [ ]:
%%time
# By looping, we make a list of lists of correlations
latout = []
for lat in fine_moisture.latitude:
    lonout = []
    latout.append(lonout)
    for lon in fine_moisture.longitude:
        val = pearsonr(
            fine_moisture.sel(latitude=lat, longitude=lon),
            bare_soil.sel(latitude=lat, longitude=lon)
        )
        try:
            # Spearman's R can fail for some values
            val += spearmanr(
                fine_moisture.sel(latitude=lat, longitude=lon),
                bare_soil.sel(latitude=lat, longitude=lon)
            )
        except ValueError:
            val += (np.nan, np.nan)
        lonout.append(val)
# Then we convert the lists to an array
arr = np.array(latout)
# And finally insert the pieces into our correlation dataset
corr.pearson_r[:] = arr[..., 0]
corr.pearson_p[:] = arr[..., 1]
corr.spearman_r[:] = arr[..., 2]
corr.spearman_p[:] = arr[..., 3]

Now that we have the dataset built, we can explore it as usual - for example, 
by plotting the r-values where significance exceeds some threshold.

In [ ]:
SIGNIFICANT = 0.05  # Choose your own!
corr.pearson_r.where(corr.pearson_p < SIGNIFICANT).plot.imshow(robust=True)

In [ ]:
corr.spearman_r.where(corr.spearman_p < SIGNIFICANT).plot.imshow(robust=True)

You now have two nice maps showing where there are significant (at your choice of `p`, eg `<0.05`) 
relationships between mean annual soil moisture and soil exposure.

* The two maps agree fairly well, which helps to make your interpretation more robust.
* The obvious next step would be interpretation: why do some areas show a 
  strong relationship, whereas others do not, and indeed a few small areas (the 
  red spots) suggest a somewhat counterintuitive increase in soil exposure with 
  increased soil moisture? 
* A sensible way to approach this would be to export the data behind these maps as netcdf
  (using `corr.to_netcdf('your_filename.nc')`), 
  import them in ArcGIS, and overlay them semi-transparently onto higher resolution 
  imagery, so that you might develop some hypotheses. For example, you might notice 
  that urbanised surfaces and forests mostly show less strong relations than do 
  grasslands, and hypothesise that grass cover disappears more quickly in response 
  to soil dryness. 
* Subsequently, you may be able to test those hypotheses in a more rigorous 
  way, perhaps by calculating the distribution of trends by land cover types, 
  using a grid-based land cover map.


## Summary and Research Ideas

This Tutorial focused on mapping the correlation between two spatiotemporal 
variables. This sort of analysis can provide important insights into the possible 
causes of observed phenomena (although correlation by itself should never be 
mistaken for causation). In this case, we demonstrated that in part of the Melbourne 
region, climate conditions (through soil moisture) are a strong influence on 
soil protective cover, and may dominate any effects of land management.

You could easily adapt the code shown here to do a similar type of correlation 
analysis but for a different pair of environmental variables, and looking at 
a different geographical region. You can also combine the trend analysis shown 
in the previous Tutorial with a correlation analysis. For example, after mapping 
trends in one environmental variable (e.g., in vegetation, tree cover, soil 
exposure, carbon uptake) you may hypothesise that those are probably due to 
another driving variable (e.g. rainfall, soil moisture, fire occurrence), and 
test that by mapping correlation.

Via [Australia's Environment Explorer](http://ausenv.online) 
you can find several environmental variables that are available in NetCDF-formatted 
data cubes. As suggested in the previous tutorial, have a look at the website, 
and see if there is anything that raises a question or research idea in your 
mind. You can also read the accompanying annual environment report for inspiration. 

Of course, there is a wide variety of other spatiotemporal data available 
online from other sources, and we can often help you find and access them if 
needed. If you have a research idea, by all means, discuss it with one of us.

The next and final tutorial will look at combining vector polygon data 
and gridded data to calculate regional statistics, another common way of processing 
and summarising gridded data.